In [520]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
import ast



In [616]:
data = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Computed_LBP_Features_Not_Including_Others_Emotion'
df = pd.read_csv(data)

print(type(df['Computed LBP'][0]))


df['Computed LBP'] = df['Computed LBP'].apply(lambda x: list(map(float, x.split(','))) if pd.notnull(x) else x)
df['Computed LBP'] = df['Computed LBP'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) else x)

print(type(df['Computed LBP'][0]))



<class 'str'>
<class 'numpy.ndarray'>


In [617]:
df.shape

(133, 8)

In [618]:
df.head()

,Subject,Filename,OnsetFrame,ApexFrame,OffsetFrame,Action Units,Estimated Emotion,Computed LBP
0,2,EP13_04,21,51,76,1,surprise,"[3.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0, 5.0, 4.0, ..."
1,2,EP11_01,16,69,106,1+L2,surprise,"[3.0, 4.0, 3.0, 2.0, 1.0, 1.0, 9.0, 1.0, 5.0, ..."
2,2,EP06_01f,56,121,155,14,repression,"[3.0, 3.0, 3.0, 2.0, 2.0, 3.0, 5.0, 5.0, 3.0, ..."
3,2,EP03_02f,16,81,136,15,repression,"[2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 9.0, 5.0, 3.0, ..."
4,2,EP02_04f,31,79,141,12+15,repression,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ..."


In [619]:
col_names = df.columns

col_names

Index(['Subject', 'Filename', 'OnsetFrame', 'ApexFrame', 'OffsetFrame',
       'Action Units', 'Estimated Emotion', 'Computed LBP'],
      dtype='object')

In [620]:

# X = df.drop(['Subject', 'Filename', 'OnsetFrame', 'ApexFrame', 'OffsetFrame',
#        'Action Units', 'Estimated Emotion'], axis=1)
# y = df['Estimated Emotion']

# X = np.array(X)

X = df['Computed LBP'].to_list()
X = np.array(X)
# X = df[['Computed LBP', 'Action Units']]

# X = np.array(df[['Computed LBP', 'Action Units']])

y = df['Estimated Emotion']


print(X.shape)



(133, 90)


In [621]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# df['Computed LBP'] = df['Computed LBP'].apply(convert_to_array)
# X_test['ApexFrame'] = X_test['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = X_train['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = np.where(X_train['ApexFrame'] == '', np.nan, X_train['ApexFrame'])

# Convert 'ApexFrame' to float, replacing NaN values with 0 (or any other desired value)
# df['Computed LBP'] = df['Computed LBP'].apply(lambda x: ' '.join(x))
# df['Computed LBP'] = df['Computed LBP'].apply(lambda x: ' '.join(x))
# X_train['Computed LBP'] = pd.to_numeric(X_train['Computed LBP'])
# X_test['Computed LBP'] = pd.to_numeric(X_test['Computed LBP'])
# X_train.flatten()


In [ ]:
from pandas import unique


print(X_test['Computed LBP'].unique())
print(X_train['Computed LBP'].unique())


In [550]:
X_train.shape, X_test.shape
y_train.shape, y_test.shape

((93,), (41,))

In [622]:
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
import joblib


# Classifier initialization
svm_classifier = SVC(kernel='linear', C=1.0)

# Training the Classifier
svm_classifier.fit(X_train, y_train)
joblib.dump(svm_classifier, '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Not_Including_Others_Emotion_90Features.joblib')


['/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Not_Including_Others_Emotion_90Features.joblib']

In [623]:
trained_SVM_Model = joblib.load('/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Not_Including_Others_Emotion_90Features.joblib')

# Test prediction using Test sample 
# predictions = trained_SVM_Model.predict(X_test)
predictions = svm_classifier.predict(X_test)

print(predictions)

accuracy = accuracy_score(y_test, predictions)
print (f"Accuracy: {accuracy * 100: .2f}%")

['disgust' 'surprise' 'repression' 'disgust' 'disgust' 'happiness'
 'surprise' 'happiness' 'repression' 'surprise' 'disgust' 'disgust'
 'disgust' 'disgust' 'disgust' 'surprise' 'disgust' 'disgust' 'disgust'
 'surprise' 'disgust' 'happiness' 'surprise' 'disgust' 'surprise'
 'repression' 'disgust' 'happiness' 'disgust' 'disgust' 'happiness'
 'disgust' 'surprise' 'disgust' 'happiness' 'disgust' 'disgust'
 'repression' 'disgust' 'disgust']
Accuracy:  35.00%


In [595]:
import cv2
from skimage import feature
import numpy as np
from sklearn.svm import SVC
import joblib

def compute_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, P=8, R=1, method="uniform")

    lbp_features_flat = lbp.flatten()[:90]

    return lbp_features_flat

def classifyME(imagePath, svmModelPath):
    image = cv2.imread(imagePath)

    lbpfeatures = compute_lbp(image)

    if len(lbpfeatures) != 90:
        print(f"Error: Expected 90 features, but got {len(lbpfeatures)} features.")
        return None

    lbpfeatures = np.array([lbpfeatures])

    trained_SVM_Model = joblib.load(svmModelPath)

    prediction = trained_SVM_Model.predict(lbpfeatures)

    return prediction


imagePathtoClassify = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Extracted_ApexFrames/EP13_06_frame109.jpg'
svm_model_path = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Including_Others_Emotion_90Features.joblib'

predictedME = classifyME(imagePathtoClassify, svm_model_path)

print(f"Micro-Expression Present is: {predictedME}")


Micro-Expression Present is: ['disgust']
